In [1]:
import xlrd
import openpyxl
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import string
import os
import natsort
warnings.filterwarnings('ignore')

In [2]:
gu_ko=['서울시 평균','강남구', '강동구', '강북구', '강서구', '관악구', '광진구', '구로구', 
       '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', 
       '서초구', '성동구', '성북구', '송파구', '양천구', '영등포구', '용산구', 
       '은평구', '종로구', '중구', '중랑구']

In [3]:
year = input("원하는 년도 입력: ")
path_dir = "오존데이터/오존_" + year +"/"
file_list = os.listdir(path_dir)

ordered_file_list = natsort.natsorted(file_list)

length = len(file_list)

for i in range(0, length):
    globals()["ozone_xlsx_{}".format(i)] = pd.read_excel(path_dir+ordered_file_list[i]) #xlsx 파일 읽기
    globals()["ozone_xlsx_{}".format(i)].drop([globals()["ozone_xlsx_{}".format(i)].index[0], globals()["ozone_xlsx_{}".format(i)].index[1], globals()["ozone_xlsx_{}".format(i)].index[2]], axis=0, inplace=True) #필요없는 행 제거
    globals()["ozone_xlsx_{}".format(i)] = globals()["ozone_xlsx_{}".format(i)].set_index("기간별 통계 ( 일별 평균 )") #index 제거
    
    file_name = path_dir+year+"_"+str(i+1)
    globals()["ozone_xlsx_{}".format(i)].to_csv(file_name+".csv", encoding='cp949', header = None) #csv로 저장
    globals()["ozone_{}".format(i+1)] = pd.read_csv(file_name+".csv", encoding='cp949') #csv로 다시 읽기

원하는 년도 입력: 2020


In [4]:
for i in range(1, length+1):
    for j in range(31, 1, -1):
        if globals()["ozone_{}".format(i)]["{}일".format(j)].isnull().sum() == len(globals()['ozone_{}'.format(i)]):
            globals()["ozone_{}".format(i)].drop(["{}일".format(j)], axis=1, inplace=True)
        else:
            break

In [5]:
for i in range(1, length+1):
    globals()["ozone_{}".format(i)].drop(["Unnamed: 1"], axis=1, inplace=True)

In [6]:
for i in range(1, length+1):
    globals()["ozone_{}".format(i)]=globals()["ozone_{}".format(i)].transpose()
    globals()["ozone_{}".format(i)]=globals()["ozone_{}".format(i)].rename(columns=globals()["ozone_{}".format(i)].iloc[0])
    globals()["ozone_{}".format(i)]=globals()["ozone_{}".format(i)] = globals()["ozone_{}".format(i)]=globals()["ozone_{}".format(i)].drop(globals()["ozone_{}".format(i)].index[0])
    globals()["ozone_{}".format(i)].reset_index(drop=True, inplace=True)

In [7]:
for i in range(1, length+1):
    globals()["ozone_{}".format(i)]['년'] = year
    globals()["ozone_{}".format(i)]['월'] = i
    globals()["ozone_{}".format(i)]['일'] = range(1, len(globals()["ozone_{}".format(i)])+1)

In [8]:
ozone = pd.DataFrame(columns=['년','월','일','오존','자치구'])

In [9]:
for i in range(1, length+1):
    average=globals()["ozone_{}".format(i)]['서울시 평균'].to_list()
    for j in range(0, len(gu_ko)):
        temp = globals()["ozone_{}".format(i)][['년','월','일',gu_ko[j]]]
        temp = temp.rename(columns={gu_ko[j]: "오존"})
        temp['오존'].fillna(value=pd.Series(average), inplace=True) #결측치 서울시 평균으로 채우기
        temp['자치구'] = gu_ko[j]
        
        ozone = ozone.append(temp)
        del temp
    
ozone.reset_index(drop=True, inplace=True)
ozone

,년,월,일,오존,자치구
0,2020,1,1,0.004,서울시 평균
1,2020,1,2,0.004,서울시 평균
2,2020,1,3,0.009,서울시 평균
3,2020,1,4,0.011,서울시 평균
4,2020,1,5,0.008,서울시 평균
...,...,...,...,...,...
9511,2020,12,27,0.013,중랑구
9512,2020,12,28,0.006,중랑구
9513,2020,12,29,0.011,중랑구
9514,2020,12,30,0.019,중랑구


In [10]:
degree = pd.DataFrame(columns=['등급'])

In [11]:
ozone = ozone.append(degree) 
o_len = len(ozone)

for i in range(0, o_len):
    if ozone['오존'][i] >= 0 and ozone['오존'][i] <= 0.030:
        ozone['등급'][i] = '좋음'
    elif ozone['오존'][i] >= 0.031 and ozone['오존'][i] <= 0.090:
        ozone['등급'][i] = '보통'
    elif ozone['오존'][i] >= 0.091 and ozone['오존'][i] <= 0.150:
        ozone['등급'][i] = '나쁨'
    elif ozone['오존'][i] >= 0.051:
        ozone['등급'][i] = '매우나쁨'

In [12]:
ozone.drop(['오존'], axis=1, inplace=True)
ozone.rename(columns={'등급': '오존'}, inplace=True)

In [13]:
ozone.to_csv("오존데이터/오존/"+year+"_오존"+".csv", encoding='cp949', index=False) # csv로 저장